In [24]:
import pandas as pd
import copy
from transformers import pipeline
sentiment_pipeline = pipeline(model = 'bhavyagiri/roberta-base-finetuned-imdb-spoilers',task='text-classification')


In [30]:
movieReviews = pd.read_json('IMDB_reviews.json', lines=True)
movieReviews['review_date'] = pd.to_datetime(movieReviews['review_date'])
movieReviews['day'] = movieReviews['review_date'].dt.day
movieReviews['month'] = movieReviews['review_date'].dt.month
movieReviews['year'] = movieReviews['review_date'].dt.year
movieReviews['review_combined'] = movieReviews['review_summary']+ ' ' + movieReviews['review_text']
movieReviews = movieReviews.drop(columns=['review_summary','review_text', 'review_date'])

review = movieReviews[movieReviews['year']==2017]
unused_columns = ['day','user_id','movie_id','month','rating','year']
review  = review.drop(unused_columns,axis=1)
review.reset_index(drop=True, inplace=True)
review = review[review['review_combined'].apply(len)<2000]

subreview = copy.deepcopy(review[0:10000])
subreview


,is_spoiler,review_combined
0,True,Awesome Classic This movie will be one of my f...
1,True,"If you a movie lover, Don't miss a second to s..."
2,True,Mona Lisa of Film We just do not get movies li...
3,True,"Very nice! Excellent movie, I have watched it ..."
4,True,Worthy of the number one spot. One of the best...
...,...,...
12629,False,Action prioritized Atomic Blonde is an action ...
12630,False,Solid action spy movie with narrative and edit...
12631,False,Thoroughly enjoyable but let down by bad accen...
12632,False,Good for killing time Good cinematography and ...


In [29]:
from tqdm import tqdm

tqdm.pandas()
subreview['sentiment'] = subreview['review_combined'].progress_apply(sentiment_pipeline)
subreview['prediction'] = subreview['sentiment'].apply(lambda x: int(x[0]['label'][-1]))
subreview['prediction_condifence'] = subreview['sentiment'].apply(lambda x: x[0]['score'])

  6%|▌         | 623/10000 [01:31<22:59,  6.80it/s]


RuntimeError: The expanded size of the tensor (634) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 634].  Tensor sizes: [1, 514]

In [28]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns


# Creating confusion matrix
cm = confusion_matrix(subreview['is_spoiler'], subreview['prediction'])

# Plotting confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=["0", "1"],
            yticklabels=["0", "1"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


KeyError: 'prediction'

In [21]:

accuracy = accuracy_score(subreview['is_spoiler'],subreview['prediction'])
avg_non_spoiler_rate = sum(subreview['is_spoiler'] == 1)/len(subreview['is_spoiler'])
false_positive_rate = cm[0][1]/(cm[0][1]+cm[0][0])
false_negative_rate = cm[1][0]/(cm[1][0]+cm[1][1])
print(f'Average non-spoiler rate: {avg_non_spoiler_rate:.2f}')
print(f'Overall accuracy: {accuracy:.2f}')
print(f'False positive rate: {false_positive_rate:.2f}')
print(f'False negative rate: {false_negative_rate:.2f}')

Average non-spoiler rate: 0.27
False positive rate: 0.01
False_negative_rate: 0.85
